Jeyner Arango 201106

Oscar Méndez 20402

# Laboratorio 9

In [5]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re
import emoji, nltk
from wordcloud import WordCloud
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from afinn import Afinn

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score

In [21]:
import numpy as np
import pandas as pd

from dash import Dash, html, jupyter_dash
import dash_core_components as dcc
import plotly.express as px

## Procesamiento de Datos

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')
english_stop_words = stopwords.words('english')
pd.set_option('display.max_colwidth', 140)
df = pd.read_csv('train.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\osjom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\osjom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def clean_tweets(tweet):
    """ 
    removemos handles @user, links https, numeros,
    pasamos a minuscula, removemos numeros, partimos texto,
    eliminamos espacios innecesarios y volvemos a unir texto 

    @return texto limpio
    """
    tweet = tweet.replace("%20", " ").replace("&amp;", "&")
    user_removed = re.sub(r'@[A-Za-z0-9]+','',tweet)
    link_removed = re.sub('https?://[A-Za-z0-9./]+','',user_removed)
    number_removed = re.sub('[^a-zA-Z]', ' ', link_removed)
    lower_case_tweet= number_removed.lower()
    tok = WordPunctTokenizer()
    words = tok.tokenize(lower_case_tweet)
    clean_tweet = (' '.join(words)).strip()
    return clean_tweet

In [9]:
def remove_stop_words(tweet):
    removed_stop_words = ' '.join([word for word in tweet.split() if word not in english_stop_words])
    return removed_stop_words

In [10]:
def lemmatize_tweet(tweet):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tweet.split()]
    lemmatized_tweet = ' '.join(lemmatized_words)
    return lemmatized_tweet

In [11]:
df['text_length'] = df['text'].apply(len)
df['text'] = df['text'].apply(clean_tweets).apply(lemmatize_tweet).apply(remove_stop_words)

In [12]:
afinn = Afinn()
df['text_sentiment'] = df['text'].apply(afinn.score)

## Analisis de Datos

In [13]:
df.head()

,id,keyword,location,text,target,text_length,text_sentiment
0,1,NaN,NaN,deed reason earthquake may allah forgive u,1,69,1.0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,38,-2.0
2,5,NaN,NaN,resident asked shelter place notified officer evacuation shelter place order expected,1,133,-1.0
3,6,NaN,NaN,people receive wildfire evacuation order california,1,65,-1.0
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfire pours school,1,88,0.0


## Dashboard

In [23]:
# Initialize the Dash app
app = Dash(__name__)

# Create a table
def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='Tweets'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    # Display the DataFrame as a table
    html.H3(children='Data Table'),
    generate_table(df),

    # Display a histogram of text lengths
    dcc.Graph(
        id='text-length-histogram',
        figure=px.histogram(df, x="text_length", nbins=20, title="Text Length Histogram")
    ),

    # Display a bar chart of text sentiment values
    dcc.Graph(
        id='text-sentiment-bar',
        figure=px.bar(df, x="id", y="text_sentiment", title="Text Sentiment Bar Chart")
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)